# Limpieza de datos

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('DelayedFlights.csv', low_memory=False)

## Redefinición de nombres de columnas
Como la base de datos tiene nombres de columnas con carácteres especiales, se realiza una estandarización de los nombres de columnas, reemplazando espacios, tildes y otros carácteres especiales que se puedan encontrar.
Esto se hace para evitar problemas futuros en el llamado de las columnas y para facilitar el trabajo del dataset.

In [16]:
df['id'] = df['Unnamed: 0']
df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(df.columns)

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'id'],
      dtype='object')


## Valores duplicados
Como se pudo observar en la fase de análisis exploratorio, aparentemente en el dataset no existen valores duplicados. Sin embargo, es importante tener en cuenta que no pueden existir 2 vuelos con el mismo número de vuelo, misma fecha de despegue, mismos origen y destino y mismo carrier. Por esta razón, se ve si existen filas que compartan toda esta información en común

In [17]:
flight_def_cols = ["FlightNum", "Year", "Month", "DayofMonth", "CRSDepTime", "Origin", "Dest", "UniqueCarrier"]
numero_inicial_filas = len(df)
print(f"Número de filas del dataset: {numero_inicial_filas}")
print(f"Número de vuelos diferentes: {len(df.drop_duplicates(subset=flight_def_cols, inplace = False))}")

Número de filas del dataset: 1936758
Número de vuelos diferentes: 1936756


Como se puede ver difieren en 2 filas, por lo cual hay 2 vuelos repetidos. Se filtran estos vuelos:

In [18]:
df.drop_duplicates(subset=flight_def_cols, inplace = True)

## Valores nulos
Como primera medida, se considera como regla dejar fuera las columnas que tienen más de un 50% de datos nulos debido a la falta de información suficiente. 

Como las columnas del dataset que tienen valores nulos son menores del 50%, se mantendrán y se procederá a imputarlos



In [19]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
cols_with_more_than50_mv = missing_values[missing_values > 50]
print(missing_values)

LateAircraftDelay    35.588892
WeatherDelay         35.588892
NASDelay             35.588892
SecurityDelay        35.588892
CarrierDelay         35.588892
ActualElapsedTime     0.433044
ArrDelay              0.433044
AirTime               0.433044
ArrTime               0.367109
TaxiIn                0.367109
TaxiOut               0.023493
CRSElapsedTime        0.010223
TailNum               0.000258
dtype: float64


Respecto a los datos de demoras, para la imputación de datos se considera que el realizarlo en base a la media de los datos puede no ser la mejor opción, ya que puede generar un sesgo en los datos. Esto considerando que la mayoría de los vuelos no presentan demoras, por lo que la media de los datos no sería representativa. Por esta razón, se opta por imputar los valores nulos con el valor 0, ya que se considera que si no hay información de demora, es porque no hubo demora en el vuelo.

Por parte de la columna TailNum, se imputa con la moda, ya que es un dato categórico y hace referencia al tipo de aviación que se utilizó en el vuelo, por lo que se considera que no se cae en un sesgo al imputar con este valor al ser el más frecuente.

fillna ArrTime by CRSArrTime + ArrDelayPor parte de la columna ArrTime, se imputa con la suma de CRSArrTime y ArrDelay, ya que se considera que si no hay información de la hora de llegada, se puede calcular con la hora de llegada programada y la demora en la llegada.

Por parte de la columna CRSElapsedTime, se imputa con la diferencia entre la hora de llegada real y la hora de salida real, ya que se considera que si no hay información de la duración del vuelo se puede calcular con estos.

Por parte de la columna ActualElapsedTime, se imputa con la diferencia entre la hora de llegada programada y la hora de salida programada, ya que se considera que si no hay información de la duración del vuelo se puede calcular con estos.

Por parte de las columnas TaxiIn y TaxiOut, se imputa con la media, ya que se considera que si no hay información del tiempo que toma al aviación llegar a la puerta de embarque o salir de ella, se puede usar como buena referencia la media de estos tiempos.

Por parte de la columna AirTime, se imputa con la diferencia entre la duración del vuelo real y el tiempo que toma al aviación llegar a la puerta de embarque y salir de ella, ya que se considera que si no hay información del tiempo que toma al aviación en el aire, se puede calcular con estos.

In [25]:
delay_fields = ['LateAircraftDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'CarrierDelay', 'ArrDelay']
df[delay_fields] = df[delay_fields].fillna(0)
df['TailNum'] = df['TailNum'].fillna(df['TailNum'].mode()[0])
df['ArrTime'] = df['ArrTime'].fillna(df['CRSArrTime'] + df['ArrDelay'])
df['CRSElapsedTime'] = df['CRSElapsedTime'].fillna(df['ArrTime'] - df['DepTime'])
df['ActualElapsedTime'] = df['ActualElapsedTime'].fillna(df['CRSArrTime'] - df['CRSDepTime'])
df['TaxiIn'] = df['TaxiIn'].fillna(df['TaxiIn'].mean())
df['TaxiOut'] = df['TaxiOut'].fillna(df['TaxiOut'].mean())
df['AirTime'] = df['AirTime'].fillna(df['ActualElapsedTime'] - df['TaxiIn'] - df['TaxiOut'])


In [27]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
print(missing_values)

Series([], dtype: float64)


# Creación de nuevas variables


Se crea la variable **DepartureHour** que indica la hora de salida del vuelo (horas), para poder analizar si la hora de salida influye en la demora del vuelo.

In [29]:
df['DepartureHour'] = (df['DepTime'] // 100).astype(int)
print(df[['DepartureHour']].head())

   DepartureHour
0             20
1              7
2              6
3             18
4             19


Se crea la variabel **ArrivalHour** que indica la hora de llegada del vuelo (horas), para poder analizar si la hora de llegada influye en la demora del vuelo.

In [13]:
df['ArrivalHour'] = (df['ArrTime'] // 100).astype(int)

print(df[['ArrivalHour']].head())

   ArrivalHour
0           22
1           10
2            8
3           19
4           21


Se crea la variable **FlightDuration** que indica la duración del vuelo en minutos, para poder analizar si la duración del vuelo influye en la demora del vuelo.

In [14]:
df['FlightDuration'] = df['ActualElapsedTime'] - df['TaxiIn'] - df['TaxiOut']
print(df[['FlightDuration']].head())


   FlightDuration
0           116.0
1           113.0
2            76.0
3            77.0
4            87.0


Se crea la variable **DistanceCategory** que indica la distancia del vuelo en categorías, para poder analizar si la distancia del vuelo influye en la demora del vuelo. Esta variable se crea a partir de la variable Distance, dividiendo los vuelos en 3 categorías: short (0-300 millas), medium (300-800 millas) y long (más de 800 millas).

In [18]:
conditions = [
    (df['Distance'] <= 300),
    (df['Distance'] <= 800),
    (df['Distance'] > 800)
]
choices = ['Short', 'Medium', 'Long']

# Crea la nueva columna 'DistanceCategory' usando np.select
df['DistanceCategory'] = np.select(conditions, choices, default='Unknown')

# Imprime las primeras filas para verificar los cambios
print(df[['Distance', 'DistanceCategory']].head(20))

    Distance DistanceCategory
0        810             Long
1        810             Long
2        515           Medium
3        515           Medium
4        688           Medium
5       1591             Long
6        828             Long
7        828             Long
8        162            Short
9       1489             Long
10      1489             Long
11       838             Long
12       220            Short
13       220            Short
14       220            Short
15       220            Short
16       220            Short
17      1093             Long
18      1093             Long
19       972             Long


Se crea la variable **DepartureDelay** que indica la demora en la salida del vuelo en minutos, para poder analizar si la demora en la salida influye en la demora del vuelo.

In [19]:
df['DepartureDelay'] = df['DepTime'] - df['CRSDepTime']
print(df[['DepTime', 'CRSDepTime', 'DepartureDelay']].head())


   DepTime  CRSDepTime  DepartureDelay
0   2003.0        1955            48.0
1    754.0         735            19.0
2    628.0         620             8.0
3   1829.0        1755            74.0
4   1940.0        1915            25.0


Se crea la variable **ArrivalDelay** que indica la demora en la llegada del vuelo en minutos, para poder analizar si la demora en la llegada influye en la demora del vuelo.

In [20]:
df['ArrivalDelay'] = df['ArrTime'] - df['CRSArrTime']
print(df[['ArrTime', 'CRSArrTime', 'ArrivalDelay']].head())

   ArrTime  CRSArrTime  ArrivalDelay
0   2211.0        2225         -14.0
1   1002.0        1000           2.0
2    804.0         750          54.0
3   1959.0        1925          34.0
4   2121.0        2110          11.0


Se crea la variable **TotalDelay** que indica la demora total del vuelo en minutos, para poder analizar si la demora total del vuelo influye en la demora del vuelo.

In [21]:
df['TotalDelay'] = df['DepartureDelay'] + df['ArrivalDelay']
print(df[['DepartureDelay', 'ArrivalDelay', 'TotalDelay']].head())

   DepartureDelay  ArrivalDelay  TotalDelay
0            48.0         -14.0        34.0
1            19.0           2.0        21.0
2             8.0          54.0        62.0
3            74.0          34.0       108.0
4            25.0          11.0        36.0


Se crea la variable **ScheduledArrivalHour** que indica la hora de llegada programada del vuelo (horas), para poder analizar si la hora de llegada programada influye en la demora del vuelo.

In [22]:
df['ScheduledArrivalHour'] = (df['CRSArrTime'] // 100).astype(int)
print(df[['CRSArrTime', 'ScheduledArrivalHour']].head())

   CRSArrTime  ScheduledArrivalHour
0        2225                    22
1        1000                    10
2         750                     7
3        1925                    19
4        2110                    21


# Guardar dataset limpio
A partir de aquí se guardará el dataset limpio y se importará en otros notebooks

In [11]:
df.to_csv('DelayedFlightsClean.csv')